# Модульный контроль №1
### Михайловской Валерии

In [2]:
import pandas as pd
import json
import numpy as np

### 1. Объедините информацию о пользователях в одной датафрейм

In [11]:
df_metrics_de = pd.read_csv('metrics_de.csv')
df_metrics_ru = pd.read_csv('metrics_ru.csv')
onboarding = pd.read_csv('onboarding.csv')

result = onboarding.merge(df_metrics_ru, on='AccountID', how='left')
result = result.merge(df_metrics_de, on='AccountID', how='left')
result.to_csv('all_data.csv')
result = result.drop(['Unnamed: 0_x'], axis=1)
result

# result['AccountOccupationText'].nunique()

# result = onboarding.merge(df_metrics_ru, on='AccountID', how='left')
# result = result.merge(df_metrics_de, on='AccountID', how='outer')
# result = result.dropna(subset=["Answers"])

,AccountID,Answers,AccountOccupationID,AccountOccupationText,AccountSystemEmailsUnsubscribe,AccountConfCrmOnCreation,AccountHadPartner,AccountWorkspaceColorType,AccountIsLiveOn10Day,AccountConfs10Day,...,BelarusTaxPortalIntegration_y,AutocompleteIntegrationBilling_y,GoogleContacts_y,ScheduledReport_y,TaskObject_y,Lang_Pl,Integration_Avito_y,Integration_Twilio,Lang_De,DadataIntegration_y
0,310340,"{""whoAmI"": ""Не хочу отвечать"", ""howMany"": ""1"",...",-35.0,Некоммерческая деятельность / Некоммерческая д...,0,0,0,0,0,"[5526, 26, 7756]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,310342,"{""whoAmI"": ""Руководитель отдела / команды"", ""h...",-70.0,Торговля / Онлайн-магазин,0,0,0,0,0,"[5526, 26, 7756]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,310344,"{""whoAmI"": ""Владелец бизнеса"", ""howMany"": ""2-5...",-124.0,Юридические услуги / Юридические услуги,0,0,0,0,0,"[3632, 26, 7756]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,310346,"{""whoAmI"": ""Не хочу отвечать"", ""howMany"": ""51-...",-1.0,IT / IT,0,0,0,0,0,"[4500, 5526, 26, 7756]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,310350,"{""whoAmI"": ""Работает самостоятельно"", ""howMany...",-1.0,IT / IT,0,0,0,0,1,"[5526, 7318, 26, 7756]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8565,333444,"{""whoAmI"": ""Работает самостоятельно"", ""howMany...",-1.0,IT / IT,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8566,333448,"{""whoAmI"": ""Работает самостоятельно"", ""howMany...",-81.0,Услуги для населения / Услуги для населения,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8567,333450,"{""whoAmI"": ""Не хочу отвечать"", ""howMany"": ""2-5...",-1.0,IT / IT,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8568,333452,"{""whoAmI"": ""Не хочу отвечать"", ""howMany"": ""51-...",NaN,NaN,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2. Информация о каком количестве пользователей представлена в данных? Какое количество видов деятельности? Постройте барплот по суммарному количеству пользователей по видам деятельности

In [4]:
onboarding_count = onboarding['AccountID'].nunique()
print('Количество пользователей в данных по онбордингу: ' + str(onboarding_count))

metrics_count = df_metrics_de['AccountID'].nunique() + df_metrics_ru['AccountID'].nunique()
print('Количество пользователей в данных метриках: ' + str(metrics_count))


Количество пользователей в данных по онбордингу: 8570
Количество пользователей в данных метриках: 9805


In [26]:
count_of_occupation = result.dropna(subset=['AccountOccupationText'])['AccountOccupationText'].nunique()
print("Количество видов детельности: " + str(count_of_occupation))
result.groupby('AccountOccupationText').sum()

Количество видов детельности: 104


,AccountID,Answers,AccountOccupationID,AccountSystemEmailsUnsubscribe,AccountConfCrmOnCreation,AccountHadPartner,AccountWorkspaceColorType,AccountIsLiveOn10Day,AccountConfs10Day,AccountConfsTrialEnd,...,BelarusTaxPortalIntegration_y,AutocompleteIntegrationBilling_y,GoogleContacts_y,ScheduledReport_y,TaskObject_y,Lang_Pl,Integration_Avito_y,Integration_Twilio,Lang_De,DadataIntegration_y
AccountOccupationText,,,,,,,,,,,,,,,,,,,,,
IT / IT,539184864,"{""whoAmI"": ""Не хочу отвечать"", ""howMany"": ""51-...",-1671.0,1,0,26,0,12455,"[4500, 5526, 26, 7756][5526, 7318, 26, 7756][7...","[4500, 5526, 7318, 26, 7756][7204, 5526, 26, 7...",...,0.0,0.0,0.0,2.0,19.0,16.0,0.0,0.0,0.0,0.0
IT / Web-разработка,88436000,"{""whoAmI"": ""Владелец бизнеса"", ""howMany"": ""2-5...",-2750.0,0,0,8,0,132,"[5526, 26, 7756][7832, 26, 7756][7444, 7450][5...","[5526, 26, 7756][7832, 26, 7756][7444, 7450][5...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IT / ИТ-аутсорсинг,55865478,"{""whoAmI"": ""Владелец бизнеса"", ""howMany"": ""51-...",-2595.0,0,0,8,0,1838,"[7444, 7450][5526, 26, 7756][5526, 26, 7756][3...","[7444, 7450][5526, 26, 7756][5526, 26, 7756][3...",...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IT / Разработка игр,19427226,"{""whoAmI"": ""Работает самостоятельно"", ""howMany...",-10560.0,0,0,0,0,1,"[5526, 26, 7756][5526, 26, 7756][5526, 26, 775...","[5526, 26, 7756][5526, 26, 7756][5526, 26, 775...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IT / Разработка программных продуктов,18663894,"{""whoAmI"": ""Работает самостоятельно"", ""howMany...",-638.0,0,0,0,0,126,"[3632, 5526, 26, 7756][3632, 26, 7756][5526, 2...","[3632, 5526, 26, 7756][3632, 26, 7756][5526, 2...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Финансовые операции / Финансовые операции,6068338,"{""whoAmI"": ""Директор / топ-менеджер"", ""howMany...",-1653.0,0,0,0,0,15,"[5526, 26, 7756][3632, 26, 7756][5526, 26, 775...","[5526, 26, 7756][3632, 26, 7756][5526, 26, 775...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Хобби, развлечения / Хобби, развлечения",6771668,"{""whoAmI"": ""Работает самостоятельно"", ""howMany...",-2940.0,0,0,0,0,0,"[5526, 26, 7756][5526, 26, 7756][5526, 26, 775...","[5526, 26, 7756][5526, 26, 7756][5526, 26, 775...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Экология / Экология,1276578,"{""whoAmI"": ""Не хочу отвечать"", ""howMany"": ""11-...",-648.0,0,0,1,0,0,"[5526, 26, 7756][5526, 5368, 26, 7756][8450, 9...","[5526, 26, 7756][5526, 5368, 26, 7756][8450, 9...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
